In [3]:
import pandas as pd
import tensorflow as tf
from transformers import TFBartForConditionalGeneration, BartTokenizerFast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
import numpy as np
from tqdm import tqdm



In [4]:
train_df = pd.read_json("summ_train.json")
train_df = train_df.dropna()
train_df = train_df[:40000]

In [5]:
test_df = pd.read_json("summ_test.json")
test_df = test_df.dropna()
test_df = test_df[:5000]

In [6]:
print("학습데이터의 개수:",len(train_df))
print("테스트데이터의 개수:",len(test_df))

학습데이터의 개수: 40000
테스트데이터의 개수: 5000


AI Hub 에서 제공하는 이 데이터는 다소 복잡한 구조를 갖고 있습니다. 학습 데이터와 테스트 데이터 각각 상위 5 개만 출력해봅시다.

In [7]:
train_df[:5]

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741778', 'category': '종합', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741792', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741793', 'category': '스포츠', 'media_..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741794', 'category': '정치', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '290741797', 'category': '종합', 'media_t..."


In [8]:
test_df[:5]

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626877', 'category': '정치', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626896', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626904', 'category': 'IT,과학', 'medi..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627450', 'category': '사회', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627465', 'category': '경제', 'media_t..."


BART 학습에 사용하기 위해서는 ‘요약하기 전의 원문’ 과’ 요약문’ 이 두 가지만 있으면 됩니다. 참고로 이 두 개의 텍스트는 위의 데이터프레임에서 ‘documents’ 열에 저장되어져 있습니다. 이를 파싱 하여 원문은’article_original’ 열에, 그리고 요약문은 ‘abstractive’ 열에 저장하는 아래의 전처리 함수 preprocess_data() 를 사용하여 새로운 데이터프레임’train_data’ 와’test_data’ 를 얻어보겠습니다.

In [9]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)
    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', ' extractive']
    return outs_df

In [10]:
train_data = preprocess_data(train_df)
train_data.head()

,media,id,article_original,abstractive,extractive
0,광양신문,290741778,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,광양신문,290741792,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
2,광양신문,290741793,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,"[3, 5, 7]"
3,광양신문,290741794,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","[2, 3, 4]"
4,광양신문,290741797,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"


In [11]:
train_data['article_original'].loc[0]

['ha당 조사료 400만원…작물별 차등 지원',
 '이성훈 sinawi@hanmail.net',
 '전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.',
 '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.',
 '올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다.',
 '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.',
 '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.',
 '농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다.',
 'ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다.',
 '벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다.',
 '논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다.',
 '전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다.',
 '최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.']

In [12]:
test_data = preprocess_data(test_df)
test_data.head()

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"[] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다., 바른미래당이 8일 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"[[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀., 지난 3일 한국이 세계 최초...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,[] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"[[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


위의 결과를 보면 현재 원문이 저장된 article_original 의 경우에는 각 문장을 원소로 하는 파이썬의 리스 트 형태로 저장되어져 있어 이를 하나의 본문으로 저장하여’news’ 열에 저장하고, train_data 의 첫번째 샘플의 news 열의 값을 출력해보겠습니다.

In [13]:
train_data['news'] = train_data['article_original'].apply(lambda x:' '.join(x))
test_data['news'] = test_data['article_original'].apply(lambda x:' '.join(x))
train_data['news'].loc[0]

'ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.'

이제 우리가 실제로 학습에 사용할 열은 train_data 에서’news’ 열과’abstractive’ 열입니다.

In [14]:
train_data[['news','abstractive']].head()

,news,abstractive
0,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


## 정수 인코딩 과정 이해하기

In [15]:
# BART 모델에 사용될 입력 데이터와 레이블을 준비하는 데이터셋 클래스를 작성합니다.

class KoBARTSummaryDataset(tf.keras.utils.Sequence):
    def __init__(self, df, tokenizer, max_len, batch_size, ignore_index=-100):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.docs = df
        self.batch_size = batch_size
        self.ignore_index = ignore_index
        self.indices = list(range(len(self.docs)))
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError("Index out of range")
        
        # 지정된 인덱스를 기반으로 해당 배치에 포함될 데이터 샘플의 인덱스 계산(이코드 죽이네. 배치크기가 2이면 0이면 0-1, 1이면 2-4...)
        batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self. batch_size]
        batch = self.docs.iloc[batch_indices]        
        
        input_ids = []
        decoder_input_ids = []
        labels = []
        
        for _, instance in batch.iterrows():
            
            # 'news' 열의 텍스트를 정수 인코딩하여 입력에 해당하는 'input_ids' 생성
            encoded_input = self.tokenizer.encode(instance['news'], max_length= self.max_len, padding='max_length', truncation=True)
            input_ids.append(encoded_input)
            
            
            # 'abstractive' 열의 텍스트를 정수 인코딩하여 레이블 생성(요약문). 디코더의 입력문
            encoded_label = self.tokenizer.encode(instance['abstractive'],max_length=self.max_len, padding='max_length', truncation=True) 
            decoder_input = [self.tokenizer.eos_token_id] + encoded_label[:-1]
            decoder_input_ids.append(decoder_input)
            
            # 레이블에  -100을 이용하여 패딩을 적용. 위에 encode_label에서 패딩으로 0인 부분을 -100으로 대체
            # todo 책에는 왜 코딩이 틀림. encoded_label은 이미 패딩이 되어 있어. 길이가 512
            # label = encoded_label + [self.ignore_index] * (self.max_len - len(encoded_label))
            # label = encoded_label + [-100] * (self.max_len - len( encoded_label))
            # label = encoded_label[:-1] + [self.tokenizer.eos_token_id] + [self.ignore_index] * (self.max_len - len(encoded_label))
            
            label = encoded_label
            label[label.index(3)] = self.tokenizer.eos_token_id #3은 패딩토큰이다.
            label = [self.ignore_index if x == 3 else x for x in label]
            labels.append(label)
            
        return {
            'input_ids': np.array(input_ids),'decoder_input_ids': np.array(decoder_input_ids), 'labels': np.array(labels)
            }
        
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

## 모델 클래스 선언

이제 모델을 선언해보겠습니다. KoBARTConditionalGeneration 클래스는 텍스트 생성을 위해 BART 모 델을 사용하며, 한국어 텍스트에 특화된 KoBART 모델을 사용합니다.

In [16]:
model = TFBartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1', from_pt=True)
tokenizer = BartTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

config.json: 0.00B [00:00, ?B/s]

c:\Users\user06\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user06\.cache\huggingface\hub\models--gogamza--kobart-base-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Xet Storage is enabled for this re

pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, yo

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [17]:
# 하이퍼파라미터   설정 
batch_size = 2
max_len = 512
lr = 3e-5
max_epochs = 2
warmup_ratio = 0.1

배치 크기는 2, 모델의 입력으로 사용할 데이터의 최대 길이는 512, 학습률 (learning rate) 는 3e‑5 입니다. 이는 0.00003 을 의미합니다. 학습 횟수에 해당하는 max_epochs 의 값은 2, warmup_ratio = 0.1 는 학습 초기에 학습률을 점진적으로 증가시키는 비율을 나타냅니다. 전체 학습 과정의 10% 동안 학습률이 점진 적으로 증가합니다.

In [18]:
# 클래스를 호출할때 __getitem__ 함수도 호출된다.

train_dataset = KoBARTSummaryDataset(train_data, tokenizer, max_len=max_len, batch_size=batch_size)
test_dataset = KoBARTSummaryDataset(test_data, tokenizer, max_len=max_len, batch_size=batch_size)

total_steps = len(train_dataset) * max_epochs
warmup_steps = int(total_steps * warmup_ratio)
total_steps = len(train_dataset) * max_epochs
warmup_steps = int(total_steps * warmup_ratio)
lr_schedule = CosineDecay(initial_learning_rate=lr, decay_steps=total_steps)
optimizer = Adam(learning_rate=lr_schedule)

In [19]:
print('첫번째 샘플의 원문 텍스트 :', train_data['news'].loc[0])

첫번째 샘플의 원문 텍스트 : ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.


In [20]:
# 여기서 끝에 0을 해주는 이유가 배치가 2라서 값이 2개씩 쌍으로 존재한다.
print('첫번째 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과 :', train_dataset[0]['input_ids'][0])

첫번째 샘플의 원문 텍스트의 정수 인코딩 및 패딩 결과 : [21582   296  9770 14666 10386 14136 16266 14476 12061 10675 10872 14165
 10001 14423 19168 13758 18482 14885   296   318   304   263   303 15195
   308   296 17761   245 26818 26102  9506 14973 18193 24873 24777 27841
 25486 14185 26379 15358 14049 18193 14871 17074 14902 15055 14634 17613
 19754 18193 14871 17074 16245 19609 10443 14291 15912 14396 11786 19609
 15410 14031 10386 12061 10675 14303 20054 14048 14355 14212 15049 14291
 14553 19609 15231 16864 16365 15828 26030 19560 16617 14130 14516 16309
 12024 14396 14355 14212 10675 19696 14491 20087 12005 14770 21598 26407
 12024 14383 16167   236 12037 16248 14195 25292 26407 15888 20066 29246
 12332 21405 16053   243 14044 20169 19304 15615 14423 14573 24884 19961
 11211 18193 19819 12333 10955 14842 15141 14497 15261 14239   296 15377
 14079 13455   240 14333 19609 17901 14355 14212 15049 19696 13590 18102
 14635 14130 14423 14576 14212 19839 15623 17287 17400 14056  9584 20466
 14537 14212 15689

In [21]:
print('정수 인코딩 및 패딩 후의  길이 :', len(train_dataset[0]['input_ids'][0]))

정수 인코딩 및 패딩 후의  길이 : 512


In [22]:
print(train_dataset[0]['decoder_input_ids'][0])

[    1 26102  9506 14973 18193 24873 24777 27841 25486 14185 26379 19609
 10443 14291 15912 14396 11786 19609 15410 14031 10386 12061 10675 14303
 20054 14048 14355 14212 15049 14291 14553 19609 15231 16864 16365 15828
 26030 19560 14063 11495 14871 17074 12147 15127 17489 15358 15272 14432
 14834 15271 15243 15869 15450 15364 14497 12332 16516 12332 23891 10586
  9879 17982 18290 15453 17210  9754 17546     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3   

In [23]:
print(train_dataset[0]['labels'][0])

[26102  9506 14973 18193 24873 24777 27841 25486 14185 26379 19609 10443
 14291 15912 14396 11786 19609 15410 14031 10386 12061 10675 14303 20054
 14048 14355 14212 15049 14291 14553 19609 15231 16864 16365 15828 26030
 19560 14063 11495 14871 17074 12147 15127 17489 15358 15272 14432 14834
 15271 15243 15869 15450 15364 14497 12332 16516 12332 23891 10586  9879
 17982 18290 15453 17210  9754 17546     1  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -

In [24]:
# 레이블을 디코딩해서 확인
testimsi = train_dataset[0]['labels'][0]
testimsi[testimsi ==-100] = 3 # array이니까 이게 가능하다. 리스트는 안됨
tokenizer.decode(testimsi)

"전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다 .</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

## 학습

In [ ]:
# best_loss는 초기값으로 무한데로 설정. 왜 무한대로 하냐면 다음 숫자는 무조건 무한대보다 적으니까 바뀔 수 밖에 없다.
best_loss = float('inf')

for epoch in range(max_epochs):
    print(f"에포크 {epoch+1}/{max_epochs}")
    
    # 여기서부터 학습
    # 누적된 손실을 저장
    total_train_loss = 0.0
    
    # 데이터를 배치 만큼 꺼내서 학습
    for batch in tqdm(train_dataset, total=len(train_dataset), desc="학습 중"):
        with tf.GradientTape() as tape:
            outputs = model(batch, training = True) # outputs이 예측값이다.
            loss = outputs.loss
            
            # 모델의 손실값으로부터 역전파를 수행하여 파라미터를 업데이트. reduce_mean쓴 것은 배치가 2개씩 실행되기 때문인듯
            total_train_loss += tf.reduce_mean(loss).numpy()
            gradients = tape.gradient(loss, model.trainable_variables) # 하나의 공식이다. 여기서 업데이트 할 미분을 구한다.
            optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # 옵티마이저가 미분값을 참고로 파라미터를 업데이트 한다.
            
    # 한 에포크 동안 손실 평균
    avg_train_loss = total_train_loss / len(train_dataset)
    print(f"훈련 손실: {avg_train_loss:.4f}")
    
    
    # 평가
    total_val_loss = 0.0

    for batch in tqdm(test_dataset, total=len(test_dataset), desc="검증 중"):
        outputs = model(batch, training=False)
        total_val_loss += tf.reduce_mean(outputs.loss).numpy()
        
    avg_val_loss = total_val_loss / len(test_dataset)
    print(f"검증 손실: {avg_val_loss:.4f}")


    # 최소 성능 모델 저장
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        # model.save_weights('best_model.weights.h5')
        # tf.saved_model.save(model, 'best_model')
        model.save_pretrained('model')
        print(f"검증 손실이 {best_loss:.4f}로 개선되었습니다. 체크포인트를 저장했습니다.")
        
    # 에코크 종료 시 데이터 셔플. 이걸 한다고 좋아진다는 보장은 없을 듯
    train_dataset.on_epoch_end()
    test_dataset.on_epoch_end()
    
print("학습종료")
    
            

## 로드 및 요약문 생성

In [ ]:
# 모델 로드
loaded_model = TFBartForConditionalGeneration.from_pretrained('model')

In [ ]:
# 요약문을 만드는 함수 구현
def summarize(text, model, tokenizer, max_length=300):
    
    # 원문을 토큰화, 패딩은 할 필요 없나보네. 학습할 때는 했는데
    inputs = tokenizer.encode(text, return_tensors="tf", max_length=512, truncation=True)
    
    # 요약문 생성
    summary_ids = model.generate(inputs, max_length=max_length, num_beams=7, repetition_penalty=2.0)
    
    # 디코딩(summary_ids가 정수로 리턴하나 부네)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

In [26]:
text = test_data.loc[25]['news']
print(text)

배우 배수지가 매니지먼트 숲과 전속계약을 체결했다. 수지는 8일 자신의 인스타그램에 '데뷔 때 부터 함께해온 소속사 JYP와 계약기간을 마치고 오늘부터 새로운 소속사 매니지먼트 숲과 함께 하게 되었다'고 밝혔다. 이어 수지는 '연습생으로 시작해서, 데뷔하고 9년의 시간이 흐른 지금까지, JYP와 함께했던 여러 영광의 순간들이 스쳐지나간다'면서 '9년 동안 항상 옆에서 서포트 해주셨던 JYP 모든 직원분들께 진심으로 감사드린다'고 인사를 잊지 않았다. 2010년 걸그룹 '미쓰에이'로 데뷔한 배수지는 2011년 KBS2 드라마 '드림하이'로 첫 연기 활동을 시작했다. 2012년 영화 '건축학개론'을 통해 스크린 데뷔를 한 뒤 가수 활동과 연기 활동을 꾸준히 병행해 오고 있다. 매니지먼트 숲 관계자는 '배우 배수지의 장점과 매력을 극대화할 수 있는 작품 선택부터 국내외 활동, 가수로서의 솔로 활동까지 활발하게 이루어질 수 있도록 지원할 예정이다'고 전했다. 특히 올해는 작품을 통해 연기자 배수지로 대중들과 만날 예정이다. 현재 촬영 중인 SBS 드라마 '배가본드'는 민항 여객기 추락 사고에 연루된 한 남자가 은폐된 진실 속에서 찾아낸 거대한 국가 비리를 파헤치게 되는 과정을 담은 이야기다. 배수지는 국정원 블랙요원 고해리 역으로 출연하며, 뒤이어 영화 '백두산'에도 합류한다. 매니지먼트 숲은 공유, 공효진, 김재욱, 서현진, 이천희, 전도연, 정유미, 남지현, 최우식, 유민규, 이재준, 정가람, 전소니 등 소속되어 있다.


In [ ]:
# 실제 테스트. 그런데 학습이 넘 시간이 많이 걸려서 중간에 중지함. 그래서 아래 코드는 작동하지 않을 것이다.
summary = summarize(text, model, tokenizer)
print(summary)